In [1]:
import sqlite3
from random import random, randint
import sys

In [2]:
# !rm example.db

# N = 10000000
# conn = sqlite3.connect('example.db')
# c = conn.cursor()
# c.execute('''CREATE TABLE test
#              (id int, x real, y real, p int, q int)''')
# gen = ((i, random(), random(), randint(0,N), randint(0,N)) for i in range(N))
# c.executemany("INSERT INTO test VALUES (?,?,?,?,?)", gen)

# conn.commit()
# conn.close()

In [3]:
conn = sqlite3.connect('example.db')
c = conn.cursor()
%time res_raw = [row for row in c.execute("SELECT id,x,y,p,q FROM test")]
conn.close()
print(sum(sys.getsizeof(r) for r in res_raw) // 1000000, 'Mb')
del res_raw

CPU times: user 9.79 s, sys: 704 ms, total: 10.5 s
Wall time: 10.5 s
800 Mb


In [4]:
conn = sqlite3.connect('example.db')
conn.row_factory = sqlite3.Row
c = conn.cursor()
%time res_raw = [row for row in c.execute("SELECT id,x,y,p,q FROM test")]
conn.close()
print(sum(sys.getsizeof(r) for r in res_raw) // 1000000, 'Mb')
del res_raw

CPU times: user 11 s, sys: 884 ms, total: 11.9 s
Wall time: 11.8 s
320 Mb


In [5]:
class RowSlot:
    __slots__ = 'id', 'x', 'y', 'p', 'q'
    def __init__(self, id, x, y, p, q):
        self.id = id
        self.x = x
        self.y = y
        self.p = p
        self.q = q

conn = sqlite3.connect('example.db')
c = conn.cursor()
%time res_slots = [RowSlot(*row) for row in c.execute("SELECT id,x,y,p,q FROM test")]
conn.close()
print(sum(sys.getsizeof(r) for r in res_slots) // 1000000, 'Mb')
del res_slots

CPU times: user 17.8 s, sys: 760 ms, total: 18.5 s
Wall time: 18.5 s
720 Mb


In [10]:
from recordclass import make_dataclass

Row = make_dataclass("Row", "id x y p q", fast_new=True)
conn = sqlite3.connect('example.db')
c = conn.cursor()
%time res = [Row(*row) for row in c.execute("SELECT id,x,y,p,q FROM test")]
conn.close()
print(sum(sys.getsizeof(r) for r in res) // 1000000, 'Mb')
del res

CPU times: user 9.97 s, sys: 740 ms, total: 10.7 s
Wall time: 10.7 s
560 Mb


In [7]:
Row(*[1, 1., 2., 3, 4])

Row(id=1, x=1.0, y=2.0, p=3, q=4)

In [8]:
Empty = make_dataclass("Empty", (), fast_new=True)


In [9]:
Empty()

Empty()